In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectPercentile, f_regression # for Univariate feature selection
from sklearn.feature_selection import RFE # Feature selection using recursive elimination

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
print('The unique data type are: ',train.dtypes.unique())

The unique data type are:  [dtype('int64') dtype('O') dtype('float64')]


In [4]:
print('The number of features in the training dataset are: ',len(train.columns))

The number of features in the training dataset are:  81


In [5]:
# Summary of column of type float
train.select_dtypes(include=['float64']).describe()

,LotFrontage,MasVnrArea,GarageYrBlt
count,1201.000000,1452.000000,1379.000000
mean,70.049958,103.685262,1978.506164
std,24.284752,181.066207,24.689725
min,21.000000,0.000000,1900.000000
25%,59.000000,0.000000,1961.000000
50%,69.000000,0.000000,1980.000000
75%,80.000000,166.000000,2002.000000
max,313.000000,1600.000000,2010.000000


In [6]:
# Summary of column of type integer
train.select_dtypes(include=['int64']).describe()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,10516.828082,6.099315,5.575342,1971.267808,1984.865753,443.639726,46.549315,567.240411,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,9981.264932,1.382997,1.112799,30.202904,20.645407,456.098091,161.319273,441.866955,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,223.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,383.500000,0.000000,477.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,712.250000,0.000000,808.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,5644.000000,1474.000000,2336.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [7]:
# Determine whether to hot enconde any of the integer variables
cutoff_criteria = 10 # A feature should have more than this number so as not be considered categorical
train_dtype_int = train.select_dtypes(include=['int64']).nunique()
print("Integer data types and their unique value counts are:\n", train_dtype_int.sort_values(axis=0))
cat_vars = train_dtype_int[train_dtype_int<10]
print('The features less than ', cutoff_criteria, 'are: \n', cat_vars)

Integer data types and their unique value counts are:
 BsmtHalfBath        3
HalfBath            3
BsmtFullBath        4
Fireplaces          4
FullBath            4
KitchenAbvGr        4
GarageCars          5
YrSold              5
PoolArea            8
BedroomAbvGr        8
OverallCond         9
OverallQual        10
TotRmsAbvGrd       12
MoSold             12
MSSubClass         15
3SsnPorch          20
MiscVal            21
LowQualFinSF       24
YearRemodAdd       61
ScreenPorch        76
YearBuilt         112
EnclosedPorch     120
BsmtFinSF2        144
OpenPorchSF       202
WoodDeckSF        274
2ndFlrSF          417
GarageArea        441
BsmtFinSF1        637
SalePrice         663
TotalBsmtSF       721
1stFlrSF          753
BsmtUnfSF         780
GrLivArea         861
LotArea          1073
Id               1460
dtype: int64
The features less than  10 are: 
 OverallCond     9
BsmtFullBath    4
BsmtHalfBath    3
FullBath        4
HalfBath        3
BedroomAbvGr    8
KitchenAbvGr    4
Fi

There are certain columns that can be assigned as datetype

In [8]:
date_cols = ['YearBuilt','YearRemodAdd','YrSold']

In [9]:
# Changing the columns from int64 to datetime
for i in date_cols:
    train[i] = pd.to_datetime(train[i])
    print(i, train[i].dtype)

YearBuilt datetime64[ns]
YearRemodAdd datetime64[ns]
YrSold datetime64[ns]


Now let's look at the values of the categorical variables

In [10]:
# train[cat_vars].unique()
cat_vars.iloc[1:10]

BsmtFullBath    4
BsmtHalfBath    3
FullBath        4
HalfBath        3
BedroomAbvGr    8
KitchenAbvGr    4
Fireplaces      4
GarageCars      5
PoolArea        8
dtype: int64

#### Columns with missing values

In [11]:
# null columns and their null counts
null_cols = train.loc[:,train.isnull().sum() != 0].isnull().sum()

In [12]:
null_cols.sort_values(axis=0)

Electrical         1
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
BsmtExposure      38
BsmtFinType2      38
GarageCond        81
GarageQual        81
GarageFinish      81
GarageType        81
GarageYrBlt       81
LotFrontage      259
FireplaceQu      690
Fence           1179
Alley           1369
MiscFeature     1406
PoolQC          1453
dtype: int64

In [13]:
print('The number of rows in the training data are: ', train.shape[0])

The number of rows in the training data are:  1460


Observe that the columns: PoolQC, MiscFeature, Alley, Fence have a lot of missing values. We'll just drop these columns to do the first pass of our analysis

In [14]:
# Drop the columns
train1 = train.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence'], axis=1)

Hell of an issue just to convert pandas dataframe to some other format in which the first column which contains the column names of training data accessible so that I can iterate over the contents to generate histogram plots to decide what kind of statistical measure should I use to impute the missing values

https://stackoverflow.com/questions/23748995/pandas-dataframe-to-list

In [15]:
null_cols_list = null_cols.index.tolist()

In [16]:
temp = train1.loc[:,null_cols_list] #data with only the 

C:\Users\Kumar\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


We observe that the columns with the most missing values have only few unique values. From here on we will work with the data with the most missing columns dropped

#### Impute missing values

To impute missing values, we'll use median for the numeric columns and most frequent value for the object columns

We use median for numeric and mode for categorical data type for imputation of missing values. <br>
https://stackoverflow.com/questions/14113187/how-do-you-set-a-conditional-in-python-based-on-datatypes

In [17]:
def impute_nas(data):           
    data.fillna(data.loc[:,data.dtypes == 'float64'].median(), inplace=True)   # impute integer columns with median
    data.fillna(data.loc[:,data.dtypes == 'int64'].median(), inplace=True)   # impute real number columns with median
    data.fillna(data.loc[:,data.dtypes == 'object'].mode().loc[0], inplace=True)   # impute categorical columns with mode

In [18]:
impute_nas(train1)

#### Create additional features using date, time columns

In [19]:
# train1['years_to_sell'] = train1['YrSold']-train1['YearBuilt']
# train['year_till_remodel'] = train1['YearRemodAdd']-train1['YearBuilt']
# train1.loc[:,['YearBuilt','YrSold','YearRemodAdd','year_till_remodel','years_to_sell']]

We observe that the date columns offer no additional information. So we will drop them

In [20]:
date_cols = train1.select_dtypes(include=['datetime64[ns]']).columns.tolist() # datetime64[ns] type columns

In [21]:
train2 = train1.select_dtypes(exclude=['datetime64[ns]'])

#### One hot encoding categorical variables

We have to encode the categorical columns with numeric values so that we can do regression <br>
One hot encoding using pandas function: <br>
https://pbpython.com/categorical-encoding.html

In [22]:
# One hot encodng of categorical columns
def one_hot_encode(data):
    cat_cols = data.select_dtypes(include=['object']).columns.tolist() ; #categorical columns as list
    return(pd.get_dummies(data, columns=cat_cols))

### Fit a regression model

In [23]:
print('No of columns in the input data prepared are:', len(train.columns.tolist()))
print('No of columns in the data prepared for linear regression are:', len(train2.columns.tolist()))

No of columns in the input data prepared are: 81
No of columns in the data prepared for linear regression are: 74


Data has to be converted from pandas dataframe to numpy so that it is usable in the linear regression function

#### Linear Regression with only numeric data

In [30]:
train3 = train1.copy()
train3 = train1.select_dtypes(exclude=['object']) #exclude categorical columns
train3 = train3.select_dtypes(exclude=['datetime64[ns]']) # remove date time data
train3.dtypes

Id                 int64
MSSubClass         int64
LotFrontage      float64
LotArea            int64
OverallQual        int64
OverallCond        int64
MasVnrArea       float64
BsmtFinSF1         int64
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
1stFlrSF           int64
2ndFlrSF           int64
LowQualFinSF       int64
GrLivArea          int64
BsmtFullBath       int64
BsmtHalfBath       int64
FullBath           int64
HalfBath           int64
BedroomAbvGr       int64
KitchenAbvGr       int64
TotRmsAbvGrd       int64
Fireplaces         int64
GarageYrBlt      float64
GarageCars         int64
GarageArea         int64
WoodDeckSF         int64
OpenPorchSF        int64
EnclosedPorch      int64
3SsnPorch          int64
ScreenPorch        int64
PoolArea           int64
MiscVal            int64
MoSold             int64
SalePrice          int64
dtype: object

In [31]:
# Split the data into training and validation set
def fit_linreg(data):
    X = data.loc[:,data.columns != 'SalePrice']
    y = data.loc[:,data.columns == 'SalePrice']
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
    regr = linear_model.LinearRegression()
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))

In [32]:
fit_linreg(train3)

Mean squared error: 1593341336.38
Variance score: 0.78


We see an immediate improvement in the variance score using only numeric data type. However to utilise as much of the data as possible we will try feature selection for the one hot encoded categorical columns

Observe that the Variance score is pretty low. Let's see if this can be improved with feature selection

### Feature selection 

This article explains the importance of Regularization when we want to chose the most useful features. https://stats.stackexchange.com/questions/287045/for-feature-selection-in-linear-regression-model-can-i-use-coefficient-estimate

Feature selection capabilities in scikit learn: https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection <br>
Example of using Univariate Feature selection: https://scikit-learn.org/stable/auto_examples/feature_selection/plot_feature_selection.html#sphx-glr-auto-examples-feature-selection-plot-feature-selection-py <br>
The Univariate Feature selection to be used for regression: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html <br>

In [35]:
def fit_linreg_feat_sel(data):
    X = data.loc[:,data.columns != 'SalePrice']
    y = data.loc[:,data.columns == 'SalePrice']
    selector =  SelectPercentile(f_regression, percentile=10)
    selector.fit(X,y)
    X = selector.transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
    regr = linear_model.LinearRegression()
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))

In [36]:
# One hot encode the categorical columns after ignoring the date time colums
train4 = train2.select_dtypes(exclude=['datetime64[ns]']) # remove date time data
train4 = one_hot_encode(train4)

In [37]:
fit_linreg_feat_sel(train4)

Mean squared error: 1210940544.32
Variance score: 0.84


C:\Users\Kumar\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Observe here that the Variance score has improved. Now let's see if we can improve upon this using a different feature selection method

#### Feature selection using a recursive method

In [38]:
def fit_linreg_feat_sel_rfe(data, no_of_features):
    X = data.loc[:,data.columns != 'SalePrice']
    y = data.loc[:,data.columns == 'SalePrice']
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
    regr = linear_model.LinearRegression()
    rfe = RFE(estimator=regr, n_features_to_select=20, step=1)
    rfe.fit(X,y)
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))

In [40]:
fit_linreg_feat_sel_rfe(train4,4)

C:\Users\Kumar\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean squared error: 877660993.49
Variance score: 0.88


### Use Random Forest

In [ ]:
from scipy import xgboost 